In [1]:
# Speed comparison SH
import pandas as pd
import numpy as np
import glob
import os

path = np.array(glob.glob('/mnt/data/20201228/zt***/Logs_full/mdLog_SH_***'))
assert(len(path) == 3)
print(path[1])
print(path[0])

logSH = pd.read_csv(path[1], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]
logSH1 = pd.read_csv(path[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]
print("original data source type: ")
print(logSH["source"].unique())
print("new data source type: ")
print(logSH1["source"].unique())
print(logSH["time"].unique())
print(logSH1["time"].unique())

logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSH1["time"] = logSH1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    
fasterPerc = []
fasterTm25 = []
fasterTm = []
fasterTm75 = []
slowerTm25 = []
slowerTm = []
slowerTm75 = []
slowerPerc = []

display(logSH['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(logSH1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())



/mnt/data/20201228/zt_7401_20201228_10.235.16.169_postvde.tar.gz/Logs_full/mdLog_SH_20201228_0923.csv
/mnt/data/20201228/zt_7401_20201228_10.235.16.168.tar.gz/Logs_full/mdLog_SH_20201228_0923.csv
original data source type: 
[23 22]
new data source type: 
[29]
['09:23:38.000' '09:23:38.630' '09:23:38.640' ... '15:39:02.000'
 '15:39:02.820' '15:39:02.950']
['09:24:06.000' '09:24:07.000' '09:24:08.000' ... '15:39:41.000'
 '15:39:44.000' '15:39:48.000']


array([1, 2, 3, 5, 6, 7])

array([1])

In [2]:
### set here ***
columns = ["StockID", "cum_volume", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]

num = 1
### set here ***
    
if num == 2:
    for i in range(0, num):
        in_dex = [16, 300, 852, 905]
        data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['source'] == np.sort(logSH['source'].unique())[i])]
        data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == np.sort(logSH1['source'].unique())[i])]
        data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
        data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
        n1 = len(data1_1["StockID"].unique())
        n2 = len(data2_1["StockID"].unique())

        print(n1)
        print(n2)
        if n1 != n2:
            sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
            data1_1 = data1_1[data1_1["StockID"].isin(sl)]
            data2_1 = data2_1[data2_1["StockID"].isin(sl)]
        for cols in ['cum_amount']:
            data1_1[cols] = data1_1[cols].round(2)
            data2_1[cols] = data2_1[cols].round(2)
        data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
        data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
        test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
        n1 = test["sequenceNo_x"].count()
        n2 = test["sequenceNo_y"].count()
        len1 = len(test)
        print(n1)
        print(n2)
        print(len1)
        print("-----------------------------------------------")
        if n2 < len1:
            display("test is not complete:")
            display(test[np.isnan(test["sequenceNo_y"])])
            print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        if (len1 == n2) & (n1 < len1):
            display("baseline is not complete:")
            display(test[np.isnan(test["sequenceNo_x"])])
            display(n2-n1)
            print((n2-n1)/n1)

        # speed check
        test1 = test[~np.isnan(test["sequenceNo_x"])]
        test1 = test1[~np.isnan(test1["sequenceNo_y"])]
        diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
        fasterPerc.append(sum(diff < 0)/len(diff))
        slowerPerc.append(sum(diff > 0)/len(diff))
        fasterTm25.append(np.quantile(abs(diff[diff < 0]), 0.25))
        fasterTm.append(np.quantile(abs(diff[diff < 0]), 0.5))
        fasterTm75.append(np.quantile(abs(diff[diff < 0]), 0.75))
        slowerTm25.append(np.quantile(abs(diff[diff > 0]), 0.25))
        slowerTm.append(np.quantile(abs(diff[diff > 0]), 0.5))
        slowerTm75.append(np.quantile(abs(diff[diff > 0]), 0.75))

        print(np.quantile(abs(diff[diff < 0]), 0.25))
        print(np.median(abs(diff[diff < 0])))
        print(np.quantile(abs(diff[diff < 0]), 0.75))
        print(np.quantile(diff[diff > 0], 0.25))
        print(np.median(diff[diff > 0]))
        print(np.quantile(diff[diff > 0], 0.75))
        print(sum(diff < 0)/len(diff))
        print(sum(diff > 0)/len(diff))
else:
    in_dex = [16, 300, 852, 905]
    data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['source'] == logSH['source'].max())]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == logSH1['source'].max())]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())

    print(n1)
    print(n2)
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    for cols in ['cum_amount']:
        data1_1[cols] = data1_1[cols].round(2)
        data2_1[cols] = data2_1[cols].round(2)
    data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
    data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
        print((n2-n1)/n1)

    # speed check
    test1 = test[~np.isnan(test["sequenceNo_x"])]
    test1 = test1[~np.isnan(test1["sequenceNo_y"])]
    diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
    fasterPerc.append(sum(diff < 0)/len(diff))
    slowerPerc.append(sum(diff > 0)/len(diff))
    fasterTm25.append(np.quantile(abs(diff[diff < 0]), 0.25))
    fasterTm.append(np.quantile(abs(diff[diff < 0]), 0.5))
    fasterTm75.append(np.quantile(abs(diff[diff < 0]), 0.75))
    slowerTm25.append(np.quantile(abs(diff[diff > 0]), 0.25))
    slowerTm.append(np.quantile(abs(diff[diff > 0]), 0.5))
    slowerTm75.append(np.quantile(abs(diff[diff > 0]), 0.75))

    print(np.quantile(abs(diff[diff < 0]), 0.25))
    print(np.median(abs(diff[diff < 0])))
    print(np.quantile(abs(diff[diff < 0]), 0.75))
    print(np.quantile(diff[diff > 0], 0.25))
    print(np.median(diff[diff > 0]))
    print(np.quantile(diff[diff > 0], 0.75))
    print(sum(diff < 0)/len(diff))
    print(sum(diff > 0)/len(diff))    
    

in_dex = [16, 300, 852, 905]
data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0)]
data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0)]

display(data1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(data2['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())


### set here ***
columns = ["StockID", "cum_volume", "close", "openPrice"]
### set here ***

data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

for cols in ["cum_amount"]:
    data1_1[cols] = data1_1[cols].round(2)
    data2_1[cols] = data2_1[cols].round(2)

test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])


# speed check
test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
fasterPerc.append(sum(diff < 0)/len(diff))
slowerPerc.append(sum(diff > 0)/len(diff))
fasterTm25.append(np.quantile(abs(diff[diff < 0]), 0.25))
fasterTm.append(np.quantile(abs(diff[diff < 0]), 0.5))
fasterTm75.append(np.quantile(abs(diff[diff < 0]), 0.75))
slowerTm25.append(np.quantile(abs(diff[diff > 0]), 0.25))
slowerTm.append(np.quantile(abs(diff[diff > 0]), 0.5))
slowerTm75.append(np.quantile(abs(diff[diff > 0]), 0.75))

print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

1454
1455
5053385
5053383
5053386
-----------------------------------------------


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount_x,close,...,ask5q,openPrice,numTrades,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange_y,time_y,cum_amount_y
337702,774481.0,1.609120e+15,4181341.0,23.0,600864,SH,94241000.0,72487167,6.608833e+08,9.44,...,6400,9.00,29959,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4947753,11201048.0,1.609138e+15,32610672.0,23.0,600085,SH,145444000.0,9475812,2.236906e+08,23.46,...,800,23.59,11882,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4979936,11269460.0,1.609139e+15,32825911.0,23.0,600733,SH,145604000.0,69869439,6.084579e+08,8.62,...,28200,8.90,36654,NaN,NaN,NaN,NaN,NaN,NaN,NaN


5.936614764163031e-07
3008.0
8008.0
14058.0
307.0
1463.0
7527.0
0.5920076891080073
0.40770299969406626


array([1, 5])

array([1])

11411
11411
11411
22104.5
51622.5
90206.5
4884.5
7930.0
11703.5
0.29024625361493295
0.709753746385067


In [3]:
import pandas as pd

path = np.array(glob.glob('/mnt/data/20201228/zt***/Logs_full/mdTradeLog***'))
assert(len(path) == 3)
print(path[1])
print(path[0])

SH = pd.read_csv(path[1], encoding="utf-8", index_col=False)
SH1 = pd.read_csv(path[0], encoding="utf-8", index_col=False)
SH = SH[SH["exchId"] == 1]
SH1 = SH1[SH1["exchId"] == 1]

import numpy as np
SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
# ####
# SH = SH[SH['TransactTime'] <= SH1['TransactTime'].max()]
# ####

columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum", 'secid', 'exchId']
# ####
# SH1 = SH1.drop_duplicates(columns, keep='first')
# ####


n1 = len(SH["SecurityID"].unique())
n2 = len(SH1["SecurityID"].unique())
print(n1)
print(n2)
print(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))
print(set(SH1["SecurityID"].unique()) - set(SH["SecurityID"].unique()))
if n1 != n2:
    sl = list(set(SH["SecurityID"].unique()) & set(SH1["SecurityID"].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)
    
# speed check
re1 = re[~np.isnan(re["sequenceNo_x"])]
re1 = re1[~np.isnan(re1["sequenceNo_y"])]
diff = re1["clockAtArrival_y"] - re1["clockAtArrival_x"]
fasterPerc.append(sum(diff < 0)/len(diff))
slowerPerc.append(sum(diff > 0)/len(diff))
fasterTm25.append(np.quantile(abs(diff[diff < 0]), 0.25))
fasterTm.append(np.quantile(abs(diff[diff < 0]), 0.5))
fasterTm75.append(np.quantile(abs(diff[diff < 0]), 0.75))
slowerTm25.append(np.quantile(abs(diff[diff > 0]), 0.25))
slowerTm.append(np.quantile(abs(diff[diff > 0]), 0.5))
slowerTm75.append(np.quantile(abs(diff[diff > 0]), 0.75))

print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

/mnt/data/20201228/zt_7401_20201228_10.235.16.169_postvde.tar.gz/Logs_full/mdTradeLog_20201228_0923.csv
/mnt/data/20201228/zt_7401_20201228_10.235.16.168.tar.gz/Logs_full/mdTradeLog_20201228_0923.csv
1454
1453
{511880}
set()
21777107
21777269
21777269
-----------------------------------------------


'baseline is not complete:'

,clockAtArrival_x,sequenceNo_x,exchId,securityType_x,__isRepeated_x,TransactTime,ChannelNo_x,ApplSeqNum,SecurityID,secid,...,TradeMoney,BidApplSeqNum,OfferApplSeqNum,clockAtArrival_y,sequenceNo_y,securityType_y,__isRepeated_y,ChannelNo_y,mdSource_y,__origTickSeq_y
21777107,NaN,NaN,1,NaN,NaN,92500020,NaN,1,603338,1603338,...,103140000,51675,156175,1609118681838912,140279,1,0,1,29,0
21777108,NaN,NaN,1,NaN,NaN,92500020,NaN,2,603338,1603338,...,2062800,116073,51656,1609118681838916,140280,1,0,1,29,1
21777109,NaN,NaN,1,NaN,NaN,92500020,NaN,3,603338,1603338,...,20628000,116073,103012,1609118681838917,140281,1,0,1,29,2
21777110,NaN,NaN,1,NaN,NaN,92500020,NaN,4,603338,1603338,...,2062800,116073,116349,1609118681838918,140282,1,0,1,29,3
21777111,NaN,NaN,1,NaN,NaN,92500020,NaN,5,603338,1603338,...,2062800,116073,119089,1609118681838919,140283,1,0,1,29,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21777264,NaN,NaN,1,NaN,NaN,92500020,NaN,158,603589,1603589,...,382440000,174580,137621,1609118681839082,140436,1,0,1,29,157
21777265,NaN,NaN,1,NaN,NaN,92500020,NaN,159,603589,1603589,...,191220000,174650,137621,1609118681839082,140437,1,0,1,29,158
21777266,NaN,NaN,1,NaN,NaN,92500020,NaN,160,603589,1603589,...,637400000,174776,137621,1609118681839084,140438,1,0,1,29,159
21777267,NaN,NaN,1,NaN,NaN,92500020,NaN,161,603589,1603589,...,254960000,151320,137621,1609118681839086,140439,1,0,1,29,160


[92500020]
2
[603338 603589]


162

640.0
1876.0
4460.0
1129.0
3447.0
15320.0
0.594201745897653
0.4055107044291971


In [4]:
fr = pd.DataFrame()
fr['cat'] = ['SSE snapshot lv2', 'SSE index', 'SSE trade']
fr['fasterPerc'] = fasterPerc
fr['25fasterTm'] = fasterTm25
fr['fasterTm'] = fasterTm
fr['75fasterTm'] = fasterTm75
fr['25slowerTm'] = slowerTm25
fr['slowerTm'] = slowerTm
fr['75slowerTm'] = slowerTm75
fr['data1'] = 'native api backup'
fr['data2'] = 'postvde'
fr = fr[['data1', 'data2', 'cat', 'fasterPerc', '25fasterTm', 'fasterTm', '75fasterTm', '25slowerTm', 'slowerTm', '75slowerTm']]
fr['fasterPerc'] = fr['fasterPerc'].apply(lambda x: '%.2f%%'% (x*100))
fr['fasterPerc'] = np.where(fr['fasterPerc'] == '100.00%', '99.99%', fr['fasterPerc'])
for cols in ['25fasterTm', 'fasterTm', '75fasterTm', '25slowerTm', 'slowerTm', '75slowerTm']:
    fr[cols] = fr[cols].astype(int)
fr

,data1,data2,cat,fasterPerc,25fasterTm,fasterTm,75fasterTm,25slowerTm,slowerTm,75slowerTm
0,native api backup,postvde,SSE snapshot lv2,59.20%,3008,8008,14058,307,1463,7527
1,native api backup,postvde,SSE index,29.02%,22104,51622,90206,4884,7930,11703
2,native api backup,postvde,SSE trade,59.42%,640,1876,4460,1129,3447,15320
